## Compare CO$_2$ HITRAN and HITEMP

⚠️ *HITEMP CO2 computations require a lot of RAM. If you're lucky to be hosted on a  https://notebooks.gesis.org binder with 8 GB RAM you should be fine. If on the others https://mybinder.org/ Binder providers with 2 GB RAM you may be at risk of crashing your Binder.* 

Compare the asymmetric stretching band  of CO2 (the 2400 cm-1 / 4.2 µm band), at various temperatures from 300 to 3500 K, in the HITRAN and HITEMP databases. If using these databases in your work, remember [to cite them](https://radis.readthedocs.io/en/latest/references/references.html#line-databases).

For any help, refer to [RADIS documentation](https://radis.readthedocs.io) 

---

First let's initialize our RADIS environment, and the calculation conditions : 

## Databases 

The HITRAN database will be automatically downloaded. The HITEMP database files for CO2 have been preconfigured in the RADIS [~/radis.json](../radis.json) config file. You can see the list of preconfigured databases with [printDatabankList()](https://radis.readthedocs.io/en/latest/gen_modules/radis.misc.config.html#radis.misc.config.printDatabankList). 


### Calculations

Let's initialize the plot environment, and choose some conditions for all the calculations : 

In [ ]:
# Setup plots : use `%matplotlib widget` for interactive plots (zoom, etc.) but graphs will not appear on download.
%matplotlib inline

from astropy import units as u
conditions = {"wavenum_min": 2000 / u.cm, 
              "wavenum_max": 2400 / u.cm,
              "molecule":'CO2',
              "isotope":'1,2,3',
              "pressure":1.01325 * u.bar,
              "mole_fraction":1,
              "path_length":1 * u.cm,
             }


Now let's calculate the spectra with [calc_spectrum()](https://radis.readthedocs.io/en/latest/source/radis.lbl.calc.html#radis.lbl.calc.calc_spectrum) 

In [ ]:
from radis import calc_spectrum
# CO2 HITRAN
s_tran = calc_spectrum(**conditions,
                       Tgas=300, 
                       databank='fetch',  # HITRAN automatically downloaded
                       name="HITRAN"
                      )
# without verbose=False this will show all the input parameters. 
# With verbose=2,3,etc... we get increasing number of details about the calculation. 

Now let's compute with HITEMP. 

⚠️ *The first computation may take a lot of time to generate .hdf5 [cache files](https://radis.readthedocs.io/en/latest/source/radis.misc.cache_files.html) from the HITEMP files. At this point this is not yet done while setting up the databases. The spectrum calculation itself will be extremelly fast for the rest of your session !*

In [ ]:
# CO2 HITEMP
s_temp = calc_spectrum(**conditions,
                       Tgas=300, 
                       databank='hitemp',  # HITEMP automatically downloaded
                       name="HITEMP2019",
                       verbose=3,          # adding some extra info for the first computation
                      )

Plot the difference with [plot_diff](https://radis.readthedocs.io/en/latest/source/radis.spectrum.compare.html#radis.spectrum.compare.plot_diff)

In [ ]:
from radis import plot_diff
plot_diff(s_temp, s_tran, Iunit='µW/cm2/sr/nm')

### Comparison for various temperatures 

Let's make a loop : 

In [ ]:
conditions.update({
    "warnings":{
            "MissingSelfBroadeningWarning": "ignore", # HITRAN has no temperature dependance for resonant-broadening
            "HighTemperatureWarning": "ignore", # HITRAN at high temperature is not valid and will trigger warnings. 
        },
    "verbose": False,  # no need to print calculation details 
})
    

for Tgas in [300, 700, 1300, 2000, 2500, 3000, 3500]:
    s_tran = calc_spectrum(**conditions,
                           Tgas=Tgas, 
                           databank='hitran',
                           name="HITRAN")
    s_temp = calc_spectrum(**conditions,
                           Tgas=Tgas, 
                           databank="hitemp",
                           name="HITEMP2019")
    plot_diff(s_temp, s_tran, 'abscoeff', title=f"{Tgas}K")